In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import scipy
import scipy.stats
import matplotlib
from scipy import interpolate
import os
import itertools
from textwrap import wrap
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
from sklearn.preprocessing import MultiLabelBinarizer
import matplotlib.font_manager as fm
import scipy.stats
import networkx as nx
from sklearn.decomposition import PCA
from sklearn import metrics
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import radius_neighbors_graph
import matplotlib.patches as mpatches
import sklearn.neighbors
from scipy.spatial import distance
from skimage.filters import unsharp_mask
from skimage.color import rgb2gray
from mpl_toolkits.axes_grid1 import ImageGrid
from collections import Counter
import skimage
from Bio import SeqIO
from Bio.KEGG import REST
from Bio.KEGG.KGML import KGML_parser
import io
import numpy as np
import gget
import matplotlib.pyplot as plt
from skimage import data, segmentation, feature, future
from sklearn.ensemble import RandomForestClassifier
from functools import partial
import sys

from mpl_toolkits.mplot3d import Axes3D
sys.path.append("../")
import utils as ut
from importlib import reload
reload(ut)


<module 'utils' from '/home/cstansbu/git_repositories/spatial_transcriptomics/clustering/spatial/../utils.py'>

In [2]:
# upload the scale factors

keys = ['ND', 'HFD8', 'HFD14']

scales = []

for key in keys:
    path = f"/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/SPT/{key}/outs/spatial/scalefactors_json.json"
    
    tmp = pd.read_json(path, lines=True)
    tmp['key'] = key
    # tmp['lowres_spot_diameter_pixels'] = tmp['spot_diameter_fullres'] * tmp['tissue_lowres_scalef']
    # tmp['lowres_spot_diameter_units'] = tmp['lowres_spot_diameter_pixels'] * 65
    
    scales.append(tmp)
    
scales = pd.concat(scales, ignore_index=True)
print(scales)
# print(scales[['key', 'spot_diameter_fullres', 'lowres_spot_diameter_pixels']])


def getscalebar(key, length, scales=scales):
    """Length is physical units """
    lookup = scales.loc[scales['key'] == key]
    pixels = lookup['spot_diameter_fullres'].values[0]
    scaleFactor = lookup['tissue_lowres_scalef'].values[0]
    units = 65 # mircons
    
    fullResPixels = (pixels * length) / units
    lowResPixels = fullResPixels * scaleFactor
    
    return lowResPixels
    
getscalebar('ND', length=100)

   spot_diameter_fullres  tissue_hires_scalef  fiducial_diameter_fullres   
0             257.195160             0.050684                 415.469120  \
1             257.368127             0.051089                 415.748513   
2             257.404844             0.049124                 415.807825   

   tissue_lowres_scalef    key  
0              0.015205     ND  
1              0.015327   HFD8  
2              0.014737  HFD14  


6.016495945974647

In [3]:
imDir = "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/images/lowres/"


lowres = {}

for f in os.listdir(imDir):
    if f.endswith("png"):
        fullpath = f"{imDir}{f}"
        img = skimage.io.imread(fullpath, as_gray=True)
        key = f.replace(".png", "")
        print(key, img.shape)
        lowres[key] = img

print('done')

HFD14 (2000, 1628)
ND (2000, 1681)
HFD8 (2000, 1692)
done


In [4]:
fPath = "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/LR/LewisLabUSCS/Mouse/Mouse-2020-Baccin-LR-pairs.xlsx"

lr = pd.read_excel(fPath, engine="openpyxl")
print(lr.shape)

lr['ligand'] = lr['Ligand.Mouse'].str.upper()
lr['receptor'] = lr['Receptor.Mouse'].str.upper()
lr.head()

(2000, 8)


,Pair.Name,Ligand.Mouse,Receptor.Mouse,Source,ManualAnnotation,Ligand.CC,Ligand.GO,Reference (PMID / KEGG),ligand,receptor
0,A2m-Lrp1,A2m,Lrp1,Ramilowski,Correct,Secreted,Other,"1702392,10652313, 12194978",A2M,LRP1
1,Adgre5-Cd55,Adgre5,Cd55,Baccin,Correct,Membrane,Other,23447688,ADGRE5,CD55
2,Adipoq-Adipor1,Adipoq,Adipor1,Ramilowski,Correct,Secreted,Other,12802337,ADIPOQ,ADIPOR1
3,Adipoq-Adipor2,Adipoq,Adipor2,Ramilowski,Correct,Secreted,Other,"12802337, 12802330",ADIPOQ,ADIPOR2
4,Adm-Calcrl&Ramp2,Adm,Calcrl&Ramp2,Baccin,Correct,Secreted,Other,10342881,ADM,CALCRL&RAMP2


In [5]:
cardOutDir = "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/CARDOuputs/"

dfList = []

for f in os.listdir(cardOutDir):
    if 'global' in f:
        fullPath = f"{cardOutDir}{f}"
        key = f.split("_")[0]
        df = pd.read_csv(fullPath)
        df = df.rename(columns={'Unnamed: 0' : 'spotId'})
        df['key'] = key
        dfList.append(df)

df = pd.concat(dfList, ignore_index=True)
print(f"{df.shape=}")
print(df['key'].value_counts())
print()
df.head()

df.shape=(7418, 13)
key
HFD8     3391
ND       2034
HFD14    1993
Name: count, dtype: int64



,spotId,B cells,Mac1,Mac3,Monocytes,T cells,NK cells,Stromal cells,Dendritic cells,Mac4,Mac2,Mac5,key
0,AAACAAGTATCTCCCA.1_ND,0.018691,0.009410,0.010737,0.070782,0.040298,0.077389,0.645332,0.015281,0.045516,0.039354,0.027211,ND
1,AAACAGCTTTCAGAAG.1_ND,0.008378,0.005700,0.038038,0.089932,0.048618,0.103983,0.575021,0.046758,0.020027,0.035451,0.028094,ND
2,AAACAGGGTCTATATT.1_ND,0.024325,0.010761,0.024323,0.117606,0.072993,0.121093,0.478383,0.026608,0.042051,0.048258,0.033598,ND
3,AAACATTTCCCGGATT.1_ND,0.023002,0.014704,0.044181,0.087019,0.053537,0.083922,0.465958,0.053223,0.087758,0.037452,0.049242,ND
4,AAACCCGAACGAAATC.1_ND,0.021954,0.003784,0.001240,0.038020,0.026386,0.061364,0.775170,0.003725,0.023106,0.025914,0.019336,ND


In [6]:
sptDir =  "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/CARDInputs/"

spt = {}

for f in os.listdir(sptDir):
    if "spt" in f and "global" in f:
        fullPath = f"{sptDir}{f}"
        key = f.split("_")[0]
        sdf = pd.read_csv(fullPath)
        sdf = sdf.rename(columns={'Unnamed: 0' : 'gene'})
        sdf = sdf.set_index('gene')
        sdf = sdf.T
        sdf.index = sdf.index.str.replace("-", ".")
        sdf = ut.normalize(sdf, 1e6)
        spt[key] = sdf
        print(f, sdf.shape)
    
print('done')

HFD8_global_spt.csv (3394, 31053)
ND_global_spt.csv (2036, 31053)
HFD14_global_spt.csv (1994, 31053)
done


In [7]:
coordDir = "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/CARDInputs/"

dfList = []

for f in os.listdir(coordDir):
    if "coord" in f and "global" in f:
        fullPath = f"{coordDir}{f}"
        key = f.split("_")[0]
        cdf = pd.read_csv(fullPath)
        cdf = cdf.rename(columns={'Unnamed: 0' : 'spotId'})
        cdf['key'] = key
        dfList.append(cdf)

cdf = pd.concat(dfList, ignore_index=True)
print(f"{cdf.shape=}")
print(cdf['key'].value_counts())
print()
cdf.head()

cdf.shape=(7424, 4)
key
HFD8     3394
ND       2036
HFD14    1994
Name: count, dtype: int64



,spotId,x,y,key
0,AAACAAGTATCTCCCA.1_HFD8,1244.795278,1189.669724,HFD8
1,AAACACCAATAACTGC.1_HFD8,405.548324,1349.579809,HFD8
2,AAACAGCTTTCAGAAG.1_HFD8,303.880251,1068.178931,HFD8
3,AAACAGGGTCTATATT.1_HFD8,344.496391,1138.478061,HFD8
4,AAACAGTGTTCCTGGG.1_HFD8,648.734268,1595.524585,HFD8


In [8]:
"""Merge CARD predictions with spatial coordinates"""
df = pd.merge(cdf, df,
              how='left', 
              left_on=['spotId', 'key'],
              right_on=['spotId', 'key'])

df = df.fillna(0)
df = df.drop_duplicates() # very important!
print(df['key'].value_counts())
df.head()

key
HFD8     3394
ND       2036
HFD14    1994
Name: count, dtype: int64


,spotId,x,y,key,B cells,Mac1,Mac3,Monocytes,T cells,NK cells,Stromal cells,Dendritic cells,Mac4,Mac2,Mac5
0,AAACAAGTATCTCCCA.1_HFD8,1244.795278,1189.669724,HFD8,0.077635,0.022735,0.104811,0.111453,0.125867,0.195064,0.059765,0.111978,0.082445,0.066571,0.041676
1,AAACACCAATAACTGC.1_HFD8,405.548324,1349.579809,HFD8,0.076987,0.041831,0.047343,0.139484,0.082711,0.093085,0.148247,0.071763,0.058744,0.142045,0.097760
2,AAACAGCTTTCAGAAG.1_HFD8,303.880251,1068.178931,HFD8,0.079216,0.043327,0.075868,0.086821,0.086695,0.097747,0.186142,0.090520,0.069670,0.094246,0.089748
3,AAACAGGGTCTATATT.1_HFD8,344.496391,1138.478061,HFD8,0.037682,0.029618,0.093281,0.109521,0.052151,0.061401,0.201332,0.094533,0.143671,0.088173,0.088637
4,AAACAGTGTTCCTGGG.1_HFD8,648.734268,1595.524585,HFD8,0.092691,0.001555,0.002745,0.075636,0.054631,0.052849,0.112912,0.008552,0.258918,0.056771,0.282740


In [9]:
def closest_node(node, nodes, metric):
    closest_index = distance.cdist([node], nodes, metric=metric).argmin()
    return closest_index

def getSubGraph(kdf, x, y, metric='minkowski', return_center=True):
    """A function to generate a graph of size n x n
    around point (x, y) """
    
    nodes = kdf[['x', 'y']]
    centerInd = closest_node([x, y], nodes, metric)
    
      
    # get the n points around that node
    nbrs = NearestNeighbors(n_neighbors=n,  
                            metric=metric, 
                            algorithm='ball_tree').fit(nodes)
    distances, indices = nbrs.kneighbors(nodes)
    
    # create the subgraph
    windowIndices = indices[centerInd]
    mask = kdf.index.isin(windowIndices)
    
    center = kdf[mask].reset_index(drop=True)
    dists = sklearn.metrics.pairwise_distances(center[['x', 'y']], metric=metric)
    
    X = dists
    
    # create the graph
    nodeMapper = dict(zip(center.index, center['spotId']))
    G = nx.from_numpy_array(X)
    G = nx.relabel_nodes(G, nodeMapper)
    
    # set positions
    pos = {}
    for idx, row in center.iterrows():
        pos[row['spotId']] = np.array(row[['x', 'y']])
    
    G.pos = pos
    
    if return_center:
        return G, center
    else:
        return G
    
    
def getFullGraph(kdf, metric='minkowski'):
    """A function to generate a graph of size n x n
    around point (x, y) """
    
    nodes = kdf[['x', 'y']]

    dists = sklearn.metrics.pairwise_distances(nodes, metric=metric)
    np.fill_diagonal(dists, 0) # no self loops!
    X = dists
    
    # create the graph
    nodeMapper = dict(zip(kdf.index, kdf['spotId']))
    G = nx.from_numpy_array(X)
    G = nx.relabel_nodes(G, nodeMapper)
    
    # set positions
    pos = {}
    for idx, row in kdf.iterrows():
        pos[row['spotId']] = np.array(row[['x', 'y']])
    
    G.pos = pos
    return G
    
    
def getKnn(kdf, k=6, metric='minkowski'):
    """A function to generate a knn graph """
    
    nodes = kdf[['x', 'y']]
      
    # get the n points around that node
    nbrs = NearestNeighbors(n_neighbors=k+1,  
                            metric=metric, 
                            algorithm='ball_tree').fit(nodes)
    distances, indices = nbrs.kneighbors(nodes)
    return indices


def knnInd2Id(kdf, knnInd):
    """Translate neighbors to spotIds """
    
    transId = {}
    
    for i, row in kdf.iterrows():
        spotId = row['spotId']
        nbrhs = kdf.iloc[knnInd[i]]['spotId'].values
        transId[spotId] = nbrhs
    return transId
    
    

def blur(sdf, transId):
    """A function to blur all columns using their neighbors.
    Expects spotIds as index """
    
    results  = []
    
    for spotId, nbhs in transId.items():
        
        # make sure the spot is present
        if spotId in sdf.index:
            nbhs = [x for x in nbhs if x in sdf.index]
            
            t = sdf.loc[nbhs].mean().to_dict()
            t['spotId'] = spotId
            results.append(t)
        
    results = pd.DataFrame(results)
    results = results.set_index('spotId')
    return results


def getCentralities(G):
    """ function to compute centralities """
    cf = pd.DataFrame({'spotId' : G.nodes()})
    cf['degree_centrality'] = cf['spotId'].map(nx.degree_centrality(G))
    cf['betweenness_centrality'] = cf['spotId'].map(nx.betweenness_centrality(G))
    # cf['closeness_centrality'] = cf['spotId'].map(nx.closeness_centrality(G))
    # cf['harmonic_centrality'] = cf['spotId'].map(nx.harmonic_centrality(G))
    cf['eigenvector_centrality'] = cf['spotId'].map(nx.eigenvector_centrality(G, tol=0.001))
    return cf


def prune(G, weight, threshold, min_components):
    """A function to prune disconnected components """
    H = nx.Graph(((u, v, e) for u,v,e in G.edges(data=True) if e[weight] < threshold))
    H.pos = G.pos
    
    for component in list(nx.connected_components(H)):
        if len(component) < min_components:
            for node in component:
                H.remove_node(node)
    return H



    
def removeDisconnectedNodes(G, radius=25, min_edges=3):
    """A function to remove spots that are not nearby
    other spots """
    H = G.copy()
    H.pos = G.pos
    
    H.remove_edges_from([(n1, n2) for n1, n2, w in H.edges(data="weight") if w > radius])
    degree = pd.DataFrame(nx.degree(H), columns=['node', 'degree'])
    
    mask = (degree['degree'] > min_edges)
    dropNodes = degree[~mask]
    
    # drop the nodes
    H.remove_nodes_from(dropNodes['node'].to_list())
    
    return H

In [10]:
keys = ['ND', 'HFD8', 'HFD14']
graph = {}
min_components = 3 # remove disconnected nodes


"""
NOTE: here we remove edges that are greater distance than the nearest neighbor
""" 
    
for i, key in enumerate(keys):
    kdf = df[df['key'] == key].reset_index(drop=True)
    G = getFullGraph(kdf)
    G = removeDisconnectedNodes(G)
    graph[key] = G
    
print('done')

done


In [11]:
cTypes = [
    'Dendritic cells', 
    'Mac4',
    'Stromal cells',
    'NK cells', 
    'Mac2', 
    'Monocytes',
    'T cells',
    'Mac5',
    'Mac3',
    'Mac1',
    'B cells',
]

In [12]:
"""Build the graph"""

allGraphs = {}
radius = 25

for i, key in enumerate(keys):
    
    kdf = df[df['key'] == key].reset_index(drop=True)
    G = graph[key]
    
    normedCols = {}
    for ctype in cTypes:
        newCol = f"{ctype} norm"
        normedCols[newCol] = ctype
        kdf[newCol] = kdf[ctype] / kdf[ctype].max()
        
    node_attr = kdf.set_index('spotId').to_dict('index')
    nx.set_node_attributes(G, node_attr)    
    
    edgelists = {}
    for u,v,e in G.edges(data=True):
        if e['weight'] < radius:
            
            for norm, ctype in normedCols.items():
                x1 = G.nodes[u][norm]
                x2 = G.nodes[v][norm]
                
                w = scipy.stats.hmean([x1, x2])
                edge = (u,v, w)
                
                if ctype in edgelists.keys():
                    edgelists[ctype].append(edge)
                else:
                    edgelists[ctype] = [edge]
                
    """ create the subgraph """
    H = nx.Graph()
    H.add_nodes_from(G.nodes())
    
    for ctype in cTypes:
        edges = edgelists[ctype]
        H.add_weighted_edges_from(edges, weight=ctype)
    H.pos = G.pos
    nx.set_node_attributes(H, node_attr)  
    allGraphs[key] = H
    
print('done')

done


In [13]:
q = 0.998 # the quantile threshold for edge existence, closer to 1 allows feweer edges

ctypes = [
    # 'B cells',
    'Dendritic cells',
    'Mac1',
    'Mac2',
    'Mac3',
    'Mac4',
    'Mac5',
    'Monocytes',
    # 'NK cells',
    # 'Stromal cells',
    'T cells'
]


keys = ['ND', 'HFD8', "HFD14"]

res = []

for key in keys:
    sdf = spt[key] # get spatial data
    G = allGraphs[key] # get the graph
    for i, ct in enumerate(ctypes):

        A = nx.to_pandas_adjacency(G, weight=ct)
        T = np.quantile(A, q)
        print(f" {key} {ct} {T:.3f}")

        # binarize the matrix
        A = pd.DataFrame(np.where(A > T, 1, 0), index=A.index, columns=A.columns)
        connected = A[A.sum() > 0].index

        mask = sdf.index.isin(connected)
        inGroup = sdf[mask].mean()
        outGroup = sdf[~mask].mean()

        lfc = (inGroup + 1) / (outGroup + 1)
        lfc = np.log2(lfc)
        lfc = pd.DataFrame(lfc.reset_index(drop=False))
        lfc.columns = ['gene', 'log2foldchange']
        lfc['key'] = key
        lfc['cellType'] = ct

        res.append(lfc)

res = pd.concat(res)
res.head()

 ND Dendritic cells 0.002
 ND Mac1 0.007
 ND Mac2 0.133
 ND Mac3 0.001
 ND Mac4 0.083
 ND Mac5 0.018
 ND Monocytes 0.153
 ND T cells 0.106
 HFD8 Dendritic cells 0.000
 HFD8 Mac1 0.000
 HFD8 Mac2 0.000
 HFD8 Mac3 0.000
 HFD8 Mac4 0.000
 HFD8 Mac5 0.000
 HFD8 Monocytes 0.000
 HFD8 T cells 0.000
 HFD14 Dendritic cells 0.033
 HFD14 Mac1 0.009
 HFD14 Mac2 0.228
 HFD14 Mac3 0.040
 HFD14 Mac4 0.111
 HFD14 Mac5 0.130
 HFD14 Monocytes 0.201
 HFD14 T cells 0.032


,gene,log2foldchange,key,cellType
0,XKR4,2.212847,ND,Dendritic cells
1,GM1992,0.000000,ND,Dendritic cells
2,GM37381,0.000000,ND,Dendritic cells
3,RP1,0.000000,ND,Dendritic cells
4,SOX17,0.330848,ND,Dendritic cells


In [14]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [21]:
lfc = 3
alpha = 0.05 # for the enrichments

pdf = res.copy()
pdf = pdf[pdf['log2foldchange'] > lfc]

# db = 'WikiPathways_2019_Mouse'
db = 'KEGG_2019_Mouse'
# db = 'ontology'

edf = []

for key in keys:
    for i, ct in enumerate(ctypes):
        kdf = pdf[(pdf['key'] == key) & (pdf['cellType'] == ct)]

        ef = gget.enrichr(kdf['gene'].unique(), database=db)
        ef = ef[ef['adj_p_val'] <= alpha]
        ef = ef[['path_name', 'overlapping_genes', 'adj_p_val']]
        ef['key'] = key
        ef['cellType'] = ct
        if len(ef) > 0:
            edf.append(ef)

edf = pd.concat(edf)
print(edf.shape)
edf.head()

Tue Jun 20 13:20:35 2023 INFO Performing Enichr analysis using database KEGG_2019_Mouse.
Tue Jun 20 13:20:41 2023 INFO Performing Enichr analysis using database KEGG_2019_Mouse.
Tue Jun 20 13:20:46 2023 INFO Performing Enichr analysis using database KEGG_2019_Mouse.
Tue Jun 20 13:20:51 2023 INFO Performing Enichr analysis using database KEGG_2019_Mouse.
Tue Jun 20 13:20:56 2023 INFO Performing Enichr analysis using database KEGG_2019_Mouse.
Tue Jun 20 13:21:03 2023 INFO Performing Enichr analysis using database KEGG_2019_Mouse.
Tue Jun 20 13:21:09 2023 INFO Performing Enichr analysis using database KEGG_2019_Mouse.
Tue Jun 20 13:21:14 2023 INFO Performing Enichr analysis using database KEGG_2019_Mouse.
Tue Jun 20 13:21:19 2023 INFO Performing Enichr analysis using database KEGG_2019_Mouse.
Tue Jun 20 13:21:30 2023 INFO Performing Enichr analysis using database KEGG_2019_Mouse.
Tue Jun 20 13:21:36 2023 INFO Performing Enichr analysis using database KEGG_2019_Mouse.
Tue Jun 20 13:21:42 2

(165, 5)


,path_name,overlapping_genes,adj_p_val,key,cellType
0,Basal transcription factors,"[TAF15, CCNH, TAF11, GTF2H1, GTF2E1, GTF2E2, T...",0.042944,ND,Dendritic cells
0,Basal transcription factors,"[TAF15, CCNH, TAF11, GTF2H1, GTF2E1, GTF2E2, T...",0.005606,ND,Mac1
0,Nucleotide excision repair,"[PCNA, CCNH, CETN2, RPA1, GTF2H1, CUL4A, POLD4...",0.031093,ND,Mac2
0,Protein processing in endoplasmic reticulum,"[SEC23A, TUSC3, HSPA4L, RPN1, NGLY1, CUL1, UBE...",0.000812,ND,Mac4
1,Basal transcription factors,"[TAF15, TAF12, CCNH, TAF11, GTF2H1, GTF2H3, GT...",0.000812,ND,Mac4


In [19]:
edf['path_name'].value_counts()

path_name
Fanconi anemia pathway                         12
Nucleotide excision repair                     10
Homologous recombination                       10
Autophagy                                       9
Basal transcription factors                     9
Pyrimidine metabolism                           8
Notch signaling pathway                         8
Amino sugar and nucleotide sugar metabolism     8
DNA replication                                 8
MAPK signaling pathway                          6
Rap1 signaling pathway                          5
Cell cycle                                      3
Ubiquitin mediated proteolysis                  2
Apoptosis                                       2
Protein processing in endoplasmic reticulum     1
NF-kappa B signaling pathway                    1
Sphingolipid metabolism                         1
Mismatch repair                                 1
Name: count, dtype: int64

In [20]:
tmp = edf[edf['cellType'] == 'Mac5']
tmp

,path_name,overlapping_genes,adj_p_val,key,cellType
0,Apoptosis,"[PIK3CD, PIK3CB, ACTB, CTSS, ACTG1, LMNB1, BBC...",0.032550,ND,Mac5
1,Protein processing in endoplasmic reticulum,"[TUSC3, NGLY1, UBE2D1, UBE2D2A, HERPUD1, CASP1...",0.032550,ND,Mac5
2,Ubiquitin mediated proteolysis,"[UBE3C, UBE2D1, CBLB, UBE2L6, UBE2D2A, UBE2L3,...",0.032550,ND,Mac5
0,Basal transcription factors,"[GTF2A1, CCNH, GTF2B, GTF2E1, GTF2E2, TAF5L, G...",0.000359,HFD8,Mac5
1,Nucleotide excision repair,"[CCNH, CETN2, XPA, XPC, POLD3, POLD4, POLD1, P...",0.000359,HFD8,Mac5
2,DNA replication,"[RNASEH2C, PRIM2, FEN1, RNASEH2B, MCM7, PRIM1,...",0.009730,HFD8,Mac5
3,Fanconi anemia pathway,"[MUS81, BLM, FAAP24, BRCA2, BRIP1, SLX4, REV3L...",0.011793,HFD8,Mac5
4,Autophagy,"[RAB7, IRS1, MTMR14, PIK3CD, IRS3, PIK3CB, HMG...",0.011793,HFD8,Mac5
5,Pyrimidine metabolism,"[DTYMK, PNP2, NUDT2, TYMS, NT5C2, NT5C3, NT5C,...",0.014888,HFD8,Mac5
6,Amino sugar and nucleotide sugar metabolism,"[TSTA3, CMAS, GALT, UXS1, HEXB, MPI, HEXA, NPL...",0.020541,HFD8,Mac5
